In [1]:
import h5py
import os
from extratorFeatures import *
import json
import shutil

In [2]:
PROGRESS_FILE = "Datasets/progressDataset.txt"
ANNOTATIONS_FOLDER = "Anotacoes_Texto/"

def salvar_progresso_imagem(caminho_imagem):
    """Salva o caminho da última imagem em que o trabalho foi iniciado."""
    progresso = {'ultimo_caminho_imagem': caminho_imagem}
    with open(PROGRESS_FILE, 'w') as f:
        json.dump(progresso, f)

def carregar_progresso_imagem():
    """Carrega o caminho da última imagem em que o trabalho foi iniciado."""
    if not os.path.exists(PROGRESS_FILE):
        return None
    with open(PROGRESS_FILE, 'r') as f:
        try:
            progresso = json.load(f)
            return progresso.get('ultimo_caminho_imagem')
        except json.JSONDecodeError:
            return None

def salvar_anotacao_texto(caminho_imagem, indice_contorno, classe):
    """Salva a anotação de um contorno em um arquivo de texto específico da imagem."""
    nome_base_imagem = os.path.basename(caminho_imagem)
    nome_arquivo_anotacao = os.path.splitext(nome_base_imagem)[0] + ".txt"
    caminho_arquivo_anotacao = os.path.join(ANNOTATIONS_FOLDER, nome_arquivo_anotacao)

    with open(caminho_arquivo_anotacao, 'a') as f:
        f.write(f"{indice_contorno},{classe}\n")

def carregar_anotacoes_anteriores(caminho_imagem):
    """Carrega os índices dos contornos já anotados para uma imagem específica."""
    nome_base_imagem = os.path.basename(caminho_imagem)
    nome_arquivo_anotacao = os.path.splitext(nome_base_imagem)[0] + ".txt"
    caminho_arquivo_anotacao = os.path.join(ANNOTATIONS_FOLDER, nome_arquivo_anotacao)

    indices_anotados = set()
    if os.path.exists(caminho_arquivo_anotacao):
        with open(caminho_arquivo_anotacao, 'r') as f:
            for linha in f:
                try:
                    indice = int(linha.strip().split(',')[0])
                    indices_anotados.add(indice)
                except (ValueError, IndexError):
                    continue
    return indices_anotados

def resetar_progresso(dataset_name='Datasets/dataset_anotado.h5'):
    """Apaga arquivos de progresso, anotações e o dataset para recomeçar do zero."""
    print("--- RESETANDO PROGRESSO ---")
    if os.path.exists(PROGRESS_FILE):
        os.remove(PROGRESS_FILE)
        print(f"Arquivo de progresso '{PROGRESS_FILE}' removido.")

    if os.path.exists(ANNOTATIONS_FOLDER):
        shutil.rmtree(ANNOTATIONS_FOLDER)
        print(f"Pasta de anotações '{ANNOTATIONS_FOLDER}' removida.")

    if os.path.exists(dataset_name):
        os.remove(dataset_name)
        print(f"Dataset HDF5 '{dataset_name}' removido.")

    print("\nProgresso de anotação resetado. Comece do início na próxima execução.")


# --- Função Principal de Anotação ---

def anotar_dataset_interativo(dataset_name='Datasets/dataset_anotado.h5', reset=False):
    """
    Cria ou atualiza um dataset de forma interativa, salvando o progresso
    em arquivos de texto por imagem.
    """
    if reset:
        resetar_progresso(dataset_name)
        return

    os.makedirs(os.path.dirname(dataset_name), exist_ok=True)
    os.makedirs(ANNOTATIONS_FOLDER, exist_ok=True)

    caminho_classe_1 = 'Frames2Treinamento/Fire/'
    caminho_classe_0 = 'Frames2Treinamento/Normal/'
    NUM_FEATURES = 15

    lista_classe_1 = [(os.path.join(caminho_classe_1, f), 'Fire') for f in sorted(os.listdir(caminho_classe_1))]
    lista_classe_0 = [(os.path.join(caminho_classe_0, f), 'Normal') for f in sorted(os.listdir(caminho_classe_0))]
    lista_completa = lista_classe_1 + lista_classe_0

    features_list = []
    labels_list = []

    ultimo_caminho = carregar_progresso_imagem()
    pular_ate_encontrar = ultimo_caminho is not None

    print("--- INICIANDO ANOTAÇÃO INTERATIVA ---")
    print("  - 's': FUMAÇA (classe 1) | 'n': NÃO FUMAÇA (classe 0)")
    print("  - 'q': SAIR e SALVAR | 'i': IGNORAR contorno")
    print(f"Dataset de saída: {dataset_name}")
    print(f"Pasta de anotações: {ANNOTATIONS_FOLDER}")
    if pular_ate_encontrar:
        print(f"Continuando a partir da imagem: {os.path.basename(ultimo_caminho)}")
    print("-" * 50)

    for i, (caminho_img, tipo_pasta) in enumerate(lista_completa):
        if pular_ate_encontrar and caminho_img != ultimo_caminho:
            continue
        elif pular_ate_encontrar and caminho_img == ultimo_caminho:
            pular_ate_encontrar = False # Encontrou, agora começa a processar
            print(f"\nRetomando anotação na imagem: {caminho_img}")
        else:
            print(f"\nProcessando nova imagem {i+1}/{len(lista_completa)}: {caminho_img}")

        img = cv2.imread(caminho_img)
        if img is None: continue

        contornos, img_hsv = extrair_contornos(img)
        if not contornos: continue

        anotacoes_feitas = carregar_anotacoes_anteriores(caminho_img)

        salvar_progresso_imagem(caminho_img)

        for j, contorno in enumerate(contornos):
            if j in anotacoes_feitas:
                print(f"  Contorno {j+1}/{len(contornos)}: Já anotado. Pulando.")
                continue

            img_display = img.copy()
            cv2.drawContours(img_display, contornos, -1, (0, 255, 0), 1)
            cv2.drawContours(img_display, [contorno], -1, (0, 255, 255), 3)

            cv2.namedWindow('Anotador - (s/n/i/q)', cv2.WINDOW_NORMAL)
            cv2.imshow('Anotador - (s/n/i/q)', img_display)
            key = cv2.waitKey(0) & 0xFF

            if key == ord('q'):
                print("\n'q' pressionado. Saindo e salvando...")
                cv2.destroyAllWindows()
                break

            label_to_save = -1
            if key == ord('s'):
                label_to_save = 1
                status = "FUMAÇA (1)"
            elif key == ord('n'):
                label_to_save = 0
                status = "NÃO FUMAÇA (0)"
            elif key == ord('i'):
                print(f"  Contorno {j+1}/{len(contornos)}: Ignorado.")
                continue
            else:
                print(f"  Contorno {j+1}/{len(contornos)}: Tecla inválida, ignorado.")
                continue

            features = extrair_features(img_hsv, contorno)
            if features is not None and features.shape[0] == NUM_FEATURES:
                features_list.append(features)
                labels_list.append(label_to_save)
                salvar_anotacao_texto(caminho_img, j, label_to_save)
                print(f"  Contorno {j+1}/{len(contornos)}: Salvo como {status}")
            else:
                print(f"  -> Aviso: Falha ao extrair features. Pulando.")

        cv2.destroyAllWindows()
        if 'key' in locals() and key == ord('q'):
            break

    if not features_list:
        print("\nNenhuma nova amostra foi anotada nesta sessão.")
        return

    X_data = np.array(features_list, dtype=np.float32)
    y_data = np.array(labels_list, dtype=np.int8)

    with h5py.File(dataset_name, 'a') as hf:
        if 'features' in hf and 'labels' in hf:
            print(f"\nAdicionando {len(X_data)} novas amostras ao dataset existente...")
            hf['features'].resize(hf['features'].shape[0] + len(X_data), axis=0)
            hf['labels'].resize(hf['labels'].shape[0] + len(y_data), axis=0)
            hf['features'][-len(X_data):] = X_data
            hf['labels'][-len(y_data):] = y_data
        else:
            print(f"\nCriando novo dataset com {len(X_data)} amostras...")
            hf.create_dataset('features', data=X_data, compression="gzip", chunks=True, maxshape=(None, NUM_FEATURES))
            hf.create_dataset('labels', data=y_data, compression="gzip", chunks=True, maxshape=(None,))

    print(f"Operação concluída com sucesso!")
    with h5py.File(dataset_name, 'r') as hf:
        print(f"Dimensões totais do dataset agora: features={hf['features'].shape}, labels={hf['labels'].shape}")


In [5]:
anotar_dataset_interativo("Datasets/dataset_features.h5", reset = False)

--- INICIANDO ANOTAÇÃO INTERATIVA ---
  - 's': FUMAÇA (classe 1) | 'n': NÃO FUMAÇA (classe 0)
  - 'q': SAIR e SALVAR | 'i': IGNORAR contorno
Dataset de saída: Datasets/dataset_features.h5
Pasta de anotações: Anotacoes_Texto/
Continuando a partir da imagem: frame_00002.jpg
--------------------------------------------------

Retomando anotação na imagem: Frames2Treinamento/Fire/frame_00002.jpg
  Contorno 1/29: Já anotado. Pulando.
  Contorno 2/29: Já anotado. Pulando.
  Contorno 3/29: Já anotado. Pulando.
  Contorno 4/29: Já anotado. Pulando.
  Contorno 5/29: Já anotado. Pulando.
  Contorno 6/29: Já anotado. Pulando.
  Contorno 7/29: Já anotado. Pulando.
  Contorno 8/29: Já anotado. Pulando.
  Contorno 9/29: Salvo como FUMAÇA (1)

'q' pressionado. Saindo e salvando...

Adicionando 1 novas amostras ao dataset existente...
Operação concluída com sucesso!
Dimensões totais do dataset agora: features=(62, 15), labels=(62,)
